### What is single-cell sequencing?
Single-cell sequencing examines the nucleic acid sequence information from individual cells with optimized next-generation sequencing technologies, providing a higher resolution of cellular differences and a better understanding of the function of an individual cell in the context of its microenvironment.

### Benefits of using single-cell sequencing
Single-cell sequencing has several advantages, including:

1. **High Resolution:** Enables the study of individual cells, providing a detailed understanding of cellular heterogeneity within a population.

2. **Precise Insights:** Unravels rare cell types or subpopulations that might be overlooked in traditional bulk sequencing methods.

3. **Accurate Profiling:** Allows for accurate characterization of cellular functions, gene expression, and genetic variations at the single-cell level.

4. **Dynamic Processes:** Captures dynamic changes within a cell population, providing insights into cellular transitions and responses over time.

5. **Clinical Relevance:** Has potential applications in personalized medicine, diagnostics, and understanding disease mechanisms at the individual cell level.

6. **Reduced Averaging Effects:** Eliminates the averaging effects seen in bulk sequencing, providing a clearer picture of the diverse molecular landscape within a sample.

7. **Cellular Heterogeneity:** Facilitates the identification of subtle differences between seemingly similar cells, enhancing our understanding of complex biological systems.

8. **Discovery of Novel Biomarkers:** Unveils new biomarkers and therapeutic targets by uncovering variations in gene expression and genomic profiles within individual cells.


### Difference between bulk and single-cell sequencing

**Table: Bulk Sequencing vs. Single-Cell Sequencing**

| Aspect                            | Bulk Sequencing                                       | Single-Cell Sequencing                                |
|-----------------------------------|-------------------------------------------------------|-------------------------------------------------------|
| **Resolution**                    | Provides an average from a population of cells         | Captures information at the individual cell level      |
| **Heterogeneity**                  | Masks cellular heterogeneity                           | Reveals cellular diversity and rare cell types         |
| **Insights into Rare Cells**      | May miss rare cells or variations                      | Enables detection and analysis of rare cells           |
| **Dynamic Processes**             | Provides a static snapshot of the entire population    | Captures dynamic changes, tracking cellular responses  |
| **Clinical Applications**         | General profiling in large populations                | Applicable to personalized medicine and diagnostics    |

**Chart: Bulk vs. Single-Cell Sequencing**

```plaintext
 100% 
  90% 
  80%                         Bulk Sequencing
  70% 
  60% 
  50% 
  40%                                   Single-Cell Sequencing
  30% 
  20% 
  10% 
   0% 
```

In the chart, you can visualize the emphasis on individual cell resolution in single-cell sequencing compared to the averaged representation in bulk sequencing. This graphical representation helps highlight the differences between the two methods.

# The Process of Single-cell Sequnecing
Single-cell sequencing is a powerful technique used in genomics to study individual cells, providing insights into the genetic information of each cell in a complex biological sample. Here's a simplified explanation of the process:

1. **Cell Isolation:**
   - Start by collecting a sample of cells from the organism or tissue you're studying. This could be blood, tissue, or any other type of sample.

2. **Single-Cell Capture:**
   - To study individual cells, you need to isolate them from the rest of the sample. This can be done using various techniques, such as microfluidics or droplet-based methods, to capture and separate each cell.

3. **Cell Lysis:**
   - Once you have isolated individual cells, you break open (lyse) each cell to release its genetic material (DNA or RNA). This step is essential to access the genetic information inside the cell.

4. **Amplification:**
   - The genetic material (DNA or RNA) from a single cell is typically very small, so it needs to be amplified or copied to generate enough material for analysis. Polymerase chain reaction (PCR) is often used for this purpose.

5. **Library Preparation:**
   - The amplified genetic material is then prepared as a library. This involves tagging the genetic material with unique identifiers, allowing you to trace it back to the individual cell it came from during analysis.

6. **Sequencing:**
   - The prepared libraries are then subjected to high-throughput sequencing machines. These machines read the genetic code of each DNA or RNA fragment, providing a massive amount of data.

7. **Data Analysis:**
   - The sequencing data is then analyzed computationally. Bioinformatic tools help interpret the data, identifying genes that are active or inactive in each individual cell. Researchers can gain insights into the diversity of cell types, gene expression patterns, and potential differences between cells.

8. **Biological Insights:**
   - Finally, the results of single-cell sequencing provide a detailed understanding of the heterogeneity within a population of cells. Researchers can discover rare cell types, identify changes in gene expression under different conditions, and gain insights into various biological processes.

Single-cell sequencing has revolutionized our ability to study cellular diversity, understand complex biological systems, and uncover new insights into health and disease. It allows scientists to go beyond average measurements and examine the specific characteristics of individual cells within a population.

### Preparation and Preprocessing

In [1]:
# import the libraries
import numpy as np # linear algebra, matrix manipulation
import scanpy as sc # computational biology
import pandas as pd # data analysis

c:\Users\Farhan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



scanpy==1.9.6 anndata==0.10.3 umap==0.5.5 numpy==1.26.3 scipy==1.11.4 pandas==2.1.4 scikit-learn==1.3.2 statsmodels==0.14.1 pynndescent==0.5.11


In [ ]:
# change the settings for the analysis
sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(dpi = 1200, facecolor='white')

In [2]:
results_file = 'write/pbmc3k.h5ad'  # the file that will store the analysis results

In [3]:
# Reading the data from the file
adata = sc.read('data/pbmc3k_raw.h5ad')

# Convert AnnData to DataFrame
adata_df = adata.to_df()

# Save DataFrame to CSV file
adata_df.to_csv('data/pbmc3k_raw.csv', index=False)

c:\Users\Farhan\AppData\Local\Programs\Python\Python311\Lib\site-packages\anndata\__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [4]:
adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`

In [5]:
adata  # testing the input of the files

AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'